In [34]:
import pandas as pd
import numpy as np
import re
from geopy.distance import geodesic
import functools

In [37]:
def insert_space(string):
    return string[0:3] + ' ' + string[3:]
def insert_dot(string):
    return string[0:2] + '.' + string[2:]

# функция, склеивающая колонки с координатами в кортеж 
def coord_merg(dataframe):
    dataframe['lat_long']=dataframe[['lat', 'lon']].apply(tuple, axis=1)
    return dataframe 

# аналогичная функция для датасета, содержащего координаты, соответствующие километрам трассы (данные первого геокодера)
def coord_km(dataframe):
    dataframe=dataframe.rename(columns={'lat_geoc': "Широта"})
    dataframe=dataframe.rename(columns={'lon_geoc': "Долгота"})
    dataframe['Широта']=round(dataframe['Широта'], 2)
    dataframe['Долгота']=round(dataframe['Долгота'], 2)
    dataframe['lat_long'] = dataframe[['Широта', 'Долгота']].apply(tuple, axis=1)
    return dataframe

# данные Глонасса
def coord_km_glonass(dataframe):
    dataframe=dataframe.rename(columns={'lat_glonass': "Широта"})
    dataframe=dataframe.rename(columns={'lon_glonass': "Долгота"})
    dataframe['Широта']=round(dataframe['Широта'], 2)
    dataframe['Долгота']=round(dataframe['Долгота'], 2)
    dataframe['lat_long'] = dataframe[['Широта', 'Долгота']].apply(tuple, axis=1)
    return dataframe

# получаем список метеостанций с координатами из поданного датасета Росгидромета
def stations_list(dataframe):
    dataframe=dataframe[['station','lat_long']]
    dataframe=dataframe.drop_duplicates()
    dataframe=dataframe.set_axis(range(0,len(dataframe)))
    return dataframe

# находим ближайшую метеостанцию, сопоставляя координаты километра трассы с координатами метеостанций из списка
def stat_km(point, stat_list):
    stations_list=stat_list
    lst=[]
    if pd.isnull(point) == True:
        lst.append(np.nan)
    else:
        for i in stations_list['lat_long']:
            x=geodesic(point, i).km
            lst.append(x)
            stations_list['dist']=pd.DataFrame(lst)
            y=stations_list['station'][stations_list['dist'] == stations_list['dist'].min()]
        y=y.to_string()
        y=re.sub("[0-9]", "", y)
        y=re.sub(" ", "", y)
        return y

In [ ]:
# данные двух геокодеров
km_coord = pd.read_csv('/home/common/emergency_datahack/dtp_data_processed/3_track/geo_data.csv')

# для примера загрузим метеорологические данные Росгидромета, в которых содержится полный список метеостанций, и сэмпл обучающей выборки
meteo = pd.read_csv("/home/common/emergency_datahack/dtp_data_processed/3_track/meteo.csv")
train = pd.read_csv("/home/common/emergency_datahack/dtp_data_processed/train.csv")

In [3]:
train=pd.read_csv("C:/Users/User/Desktop/dtp_data_processed/train.csv")
km_coord=pd.read_csv("C:/Users/User/Desktop/Glonass/Yandex/geo_data.csv")
#raw_path="C:/Users/User/Desktop/Glonass/Исходники/"
#R21_met = pd.read_csv(raw_path+"P-21_meteo.csv", encoding='cp1251', sep=';')
#M8_met = pd.read_csv(raw_path+"М-8_meteo.csv", encoding='cp1251', sep=';')
meteo = pd.read_csv("C:/Users/User/Desktop/Glonass/Исходники/meteo.csv")

C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (16,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# удаляем наблюдения по трассе М-4
train=train[(train['road_id'] !=5)]

# используем данные первого геокодера
km_coord=km_coord.dropna(subset=['lat_geoc'])
km_coord=km_coord.dropna(subset=['lon_geoc'])
km_coord=coord_km(km_coord)
km_coord=km_coord[['road_id', 'road_km', 'lat_long']]
km_coord.head()

,road_id,road_km,lat_long
0,9,0,"(55.76, 37.61)"
1,9,1,"(55.76, 37.62)"
2,9,2,"(55.76, 37.63)"
3,9,3,"(55.77, 37.64)"
4,9,4,"(55.77, 37.63)"


In [5]:
# добавляем координаты в тренировочную выборку
train_cord=pd.merge(train,km_coord,on=["road_id", "road_km"],how='left')
train_cord.head()

,lon,lat,datetime,road_id,road_km,man_injured_num,man_dead_num,car_damaged_num,data_source,road_name,target,dup_group,lat_long
0,NaN,NaN,2016-06-28 12:00:00,9,1223,0,0,1.0,gibdd,М-8 (E115) =Холмогоры= Москва-Архангельск че...,1,NaN,NaN
1,NaN,NaN,2016-07-02 08:00:00,9,27,0,0,2.0,gibdd,М-8 (E115) =Холмогоры= Москва-Архангельск че...,1,NaN,"(55.95, 37.82)"
2,NaN,NaN,2016-10-22 08:00:00,9,29,0,0,2.0,gibdd,М-8 (E115) =Холмогоры= Москва-Архангельск че...,1,NaN,"(55.97, 37.84)"
3,0.000000,0.000000,2012-08-19 06:00:00,9,217,1,5,2.0,gochs,М-8 (E115) =Холмогоры= Москва-Архангельск че...,2,NaN,"(57.26, 39.49)"
4,37.898809,56.056055,2019-12-16 14:00:00,9,29,0,0,2.0,gibdd,М-8 (E115) =Холмогоры= Москва-Архангельск че...,1,NaN,"(55.97, 37.84)"


In [6]:
# создаем полный список метеостанций для обеих трасс
seed = 43

meteo_sam=meteo.groupby("station").sample(frac=0.001, random_state=seed)
meteo_sam.loc[meteo_sam['station']=='MOSKBAL', 'lat'] = 55.8 # вручную добавляем координаты для станций 'MOSKBAL' и 'MONCHEG', взятые из других датасетов Росгидромета
meteo_sam.loc[meteo_sam['station']=='MOSKBAL', 'lon'] = 37.5
meteo_sam.loc[meteo_sam['station']=='MONCHEG', 'lat'] = 67.9 
meteo_sam.loc[meteo_sam['station']=='MONCHEG', 'lon'] = 32.9

coord_merg(meteo_sam)
full_stations_list=stations_list(meteo_sam)
full_stations_list

,station,lat_long
0,ALEKSAN,"(56.4, 38.75)"
1,APATIT,"(67.55, 33.35)"
2,ARHAN01,"(64.61666388888891, 40.516663888888885)"
3,BABAEVO,"(59.4, 35.93)"
4,BIRYKOV,"(59.58, 41.48)"
5,DANIL29,"(58.2, 40.18)"
6,DVINSKI,"(62.82833333333333, 42.791944444444454)"
7,EMECK,"(63.48, 41.79583333333333)"
8,HOLMOGO,"(64.22388888888891, 41.64666666666667)"
9,KALEVAL,"(65.22, 31.15)"


In [7]:
# пример работы на выборке из train.csv, т.к. выполнение занимает время
seed=1
train_cord_sam=train_cord.sample(frac=0.03, random_state=seed)
train_cord_sam=train_cord_sam.reset_index(drop=True)

In [36]:
train_cord_sam['station'] = train_cord_sam['lat_long'].map(functools.partial(stat_km, stat_list=full_stations_list))
train_cord_sam['station'] = train_cord_sam['station'].where(pd.notnull(train_cord_sam['station']), np.nan) # т.к. функиця возвращает None для пропусков, вручную меняем на NaN
train_cord_sam.head()

,lon,lat,datetime,road_id,road_km,man_injured_num,man_dead_num,car_damaged_num,data_source,road_name,target,dup_group,lat_long,station
0,40.109196,59.385070,2019-07-27 12:00:00,9,488,1,0,3.0,gibdd,М-8 (E115) =Холмогоры= Москва-Архангельск че...,2,NaN,"(59.38, 40.11)",VOLOGDA
1,37.753100,55.898600,2019-01-02 14:00:00,9,18,0,0,2.0,gibdd,М-8 (E115) =Холмогоры= Москва-Архангельск че...,1,NaN,"(55.89, 37.73)",MOSKVDN
2,NaN,NaN,2017-06-13 16:00:00,9,32,0,0,3.0,gibdd,М-8 (E115) =Холмогоры= Москва-Архангельск че...,1,NaN,"(55.98, 37.87)",MOSKVDN
3,34.075234,63.777879,2020-08-05 16:00:00,14,726,0,0,4.0,gibdd,М-18 ( E105) =Кола= С.-Петербург-Мурм...,1,NaN,"(63.78, 34.08)",SEGEJA
4,NaN,NaN,2016-10-29 01:00:00,9,198,0,0,1.0,gibdd,М-8 (E115) =Холмогоры= Москва-Архангельск че...,1,NaN,"(57.14, 39.34)",ROSTOV
